In [28]:
import geopandas as gpd
import pandas as pd

def glac_vectors(glacierID,terminus_data,terminus_centerline,terminus_sidelines, start_date='1984-12-31', end_date='2022-12-31',glacierid_column='GlacierID',date_column='Date'):
    
    #If the data is not a dataframe, open it adn replace the variable with the data
    if not isinstance(terminus_data, gpd.GeoDataFrame):
        terminus_data = gpd.read_file(terminus_data)
    if not isinstance(terminus_centerline, gpd.GeoDataFrame):
        terminus_centerline = gpd.read_file(terminus_centerline)
    if not isinstance(terminus_sidelines, gpd.GeoDataFrame):
        terminus_sidelines = gpd.read_file(terminus_sidelines)
    
    #set up the terminus data
    glac = terminus_data[terminus_data[glacierid_column] == glacierID]
    
    glac = glac[glac[date_column]>start_date]
    glac = glac[glac[date_column]<end_date]
    
    glac['Datetime'] = pd.to_datetime(glac[date_column])
    
    #set up the other layers
    sideline=terminus_sidelines[terminus_sidelines[glacierid_column]==glacierID]
    center = terminus_centerline[terminus_centerline[glacierid_column]==glacierID]
    
    return glac,center,sideline


In [29]:
glac,center,sideline = glac_vectors(2,terminus_data = '../Documents/PGC/Terminus/seelcted_glaciers_combined/selected_glacs_clipped_edit.shp',
                                    terminus_centerline='../Documents/PGC/Centerlines/termpicks_centerlines/termpicks_centerlines.shp',terminus_sidelines='../Documents/PGC/Glacier_outlines/side_lines/sidelines_v1.shp')

In [30]:
glac

,level_0,level_1,Date,Glacier ID,Satellite,Error,GlacierID,Year,Month,Day,...,DecDate,ImageID,Center_X,Center_Y,Id,layer,path,length,geometry,Datetime
12701,3871,0,2011-06-29,None,Landsat,None,2,2011,06,29,...,2011.490297,LE07_L1TP_013009_20110629_20161207_01_T1,-52.409879,-52.409879,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,3861.004643,"LINESTRING Z (-258717.778 -1978508.389 0.000, ...",2011-06-29
12809,2653,0,2000-03-17,GID2,Landsat7,26.18665158,2,2000,03,17,...,2000.207536,None,NaN,NaN,NaN,repro_GID2,C:/Users/sofyg/Documents/PGC/Terminus/Zhang/se...,3821.842208,"LINESTRING Z (-259616.783 -1982557.247 0.000, ...",2000-03-17
12842,3873,0,2011-07-08,None,Landsat,None,2,2011,07,08,...,2011.514954,LE07_L1TP_012010_20110708_20161207_01_T1,-52.408369,-52.408369,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,3806.013980,"LINESTRING Z (-258609.566 -1978250.925 0.000, ...",2011-07-08
12947,3624,0,2008-07-29,None,Landsat 7,None,2,2008,07,29,...,2008.573657,LE07_L1TP_014009_20080729_20161225_01_T1,-52.422824,-52.422824,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,3763.882032,"LINESTRING Z (-257951.109 -1981188.235 0.000, ...",2008-07-29
13092,3872,0,2011-07-01,None,Landsat,None,2,2011,07,01,...,2011.495776,LE07_L1TP_013009_20110629_20161207_01_T1,-52.406777,-52.406777,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,3704.503953,"LINESTRING Z (-258668.169 -1978500.452 0.000, ...",2011-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18397,4109,0,1989-03-20,None,Landsat 5,None,2,1989,03,20,...,1989.213584,0,-52.502746,-52.502746,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,2188.151092,"LINESTRING Z (-261827.522 -1981396.885 0.000, ...",1989-03-20
18492,4126,0,1992-03-21,None,Landsat 5,None,2,1992,03,21,...,1992.218465,0,-52.505233,-52.505233,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,2105.144250,"LINESTRING Z (-261884.986 -1981440.728 0.000, ...",1992-03-21
18502,4134,0,1994-09-26,None,Landsat 5,None,2,1994,09,26,...,1994.734132,0,-52.502887,-52.502887,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,2065.430109,"LINESTRING Z (-261723.471 -1981317.498 0.000, ...",1994-09-26
18521,4112,0,1989-09-05,None,Landsat 5,None,2,1989,09,05,...,1989.676598,0,-52.506445,-52.506445,NaN,selected_termpick_glacs,C:/Users/sofyg/Documents/PGC/Terminus/termpick...,1998.996588,"LINESTRING Z (-261925.176 -1981471.392 0.000, ...",1989-09-05
